In [1]:
import os
import numpy as np
import uproot
import yaml
import re
import math
import ROOT
import copy
import concurrent.futures
from tqdm import tqdm

Welcome to JupyROOT 6.22/06


# plot_asym helper code

In [2]:
def print_yaml_structure(data, indent=0):
    for key, value in data.items():
        print(" " * indent + key + ":")
        if isinstance(value, dict):
            print_yaml_structure(value, indent+2)
        else:
            if(type(value)==list):
                value="<LIST>"
            print(" " * (indent+2) + str(value))

In [3]:
def extract_numbers(string):
    pattern = r"-?\d+(?:\.\d+)?"
    numbers = re.findall(pattern, string)
    return [float(number) for number in numbers]

In [15]:
def get_directories_with_values(directory_path, x_values):
    # get a list of directories in the given path
    directories = os.listdir(directory_path)
    
    # filter the list to only include directories that contain every element in x_values
    filtered_directories = []
    for d in directories:
        # Sometimes, brufit rounds the bin weirdly, so I do this to make sure we stil get it
        # Be careful though if the binning is really tiny!
        if (all(str(x) in d for x in x_values)) or (all(str(np.round(x-0.01,2)) in d for x in x_values)) or (all(str(np.round(x+0.01,2)) in d for x in x_values)):
            filtered_directories.append(d)
    
    return filtered_directories

In [5]:
def read_bruout(result_file):
    param_names = []
    param_values = []
    param_errors = []
    
    with uproot.open(result_file) as f:
        if 'ResultTree' in f:
            tree = f['ResultTree']

            # Loop over all branches in the tree
            for branch_name in tree.keys():

                # Skip the error branches
                if branch_name.endswith('_err'):
                    continue
                if(branch_name in ["NLL"]):
                    continue
                # Get the parameter name and its value
                param_name = branch_name
                param_value = tree[param_name].array()[0]

                # Get the corresponding error branch
                err_branch_name = f'{param_name}_err'
                if err_branch_name not in tree:
                    continue

                # Get the parameter error
                param_error = tree[err_branch_name].array()[0]

                # Append the data to the arrays
                if param_name not in param_names:
                    param_names.append(param_name)
                    param_values.append([float(param_value)])
                    param_errors.append([float(param_error)])
                else:
                    index = param_names.index(param_name)
                    param_values[index].append(float(param_value))
                    param_errors[index].append(float(param_error))
    return param_names, param_values, param_errors

In [11]:
def read_splot(subdir):
    relpath = subdir.split('/')[-1]
    
    x_values = extract_numbers(relpath)

    # Open the TTree called ResultTree
    result_file = os.path.join(subdir, 'ResultsHSMinuit2.root')
    if(not os.path.exists(result_file)):
        return None,None,None,None
    param_names, param_values, param_errors = read_bruout(result_file)
    return x_values , param_names, param_values, param_errors

In [13]:
def read_sideband(subdir=""):
    relpath = subdir.split('/')[-1]
    
    x_values = extract_numbers(relpath)
    
    def calculate_means(lst):
        result = []
        for i in range(0, len(lst), 2):
            mean = (lst[i] + lst[i+1]) / 2
            result.append(np.round(mean,2))
        return result
    
    x_values = calculate_means(x_values)
    
    # Open the TTree
    result_file = os.path.join(subdir, 'sideband.root')
    if(not os.path.exists(result_file)):
        return None,None,None,None
     # open the result file with PyROOT
    root_file = ROOT.TFile(result_file)
    # access the TVectorT<double> object
    purity_vector = root_file.Get("purity_4")
    purity = purity_vector[0]
    root_file.Close()
    
    # Now get the sigbg sideband data
    sigbg_file="/".join(subdir.split('/')[:-2])+"/outObsBins_sdbnd_sigbg"
    sigbg_file+="/"+get_directories_with_values(sigbg_file, x_values)[0]+"/ResultsHSMinuit2.root"
    if(not os.path.exists(sigbg_file)):
        return None,None,None,None
    param_names_sigbg, param_values_sigbg, param_errors_sigbg = read_bruout(sigbg_file)
    
    # Now get the bg sideband data
    bg_file="/".join(subdir.split('/')[:-2])+"/outObsBins_sdbnd_bg"
    bg_file+="/"+get_directories_with_values(bg_file, x_values)[0]+"/ResultsHSMinuit2.root"
    if(not os.path.exists(bg_file)):
        return None,None,None,None
    param_names_bg, param_values_bg, param_errors_bg = read_bruout(bg_file)

    # Return
    param_names = param_names_sigbg
    param_values = []
    param_errors = []
    for pn,pv_sigbg,pv_bg,pe_sigbg,pe_bg in zip(param_names,param_values_sigbg,param_values_bg,
                                               param_errors_sigbg,param_errors_bg):
        if("Yld" in pn):
            continue
        
        param_values.append([pv_sigbg[0]/purity-(1-purity)*pv_bg[0]/purity])
        param_errors.append([math.sqrt((pe_sigbg[0]/purity)**2+((1-purity)*pe_bg[0]/purity)**2)])
    
    
    return [round(float(x),2) for x in x_values], param_names, param_values, param_errors
# subdir="/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/asym/Fall2018_RGA_inbending/cut_v6/piplus_pi0/z/ML/AZI/outSdbndBins/z_0.620000_0.700000"
# read_sideband(subdir)

# f=ROOT.TFile("/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/asym/Fall2018_RGA_inbending/cut_v6/piplus_pi0/z/ML/AZI/outSdbndBins/z_0.620000_0.700000/sideband.root")
# fit=f.Get("f_sdbnd")
# hnorm=f.Get("Mdiphoton_normed")
# c=ROOT.TCanvas()
# fit.Draw("")
# hnorm.Draw("same")
# c.Draw()

In [21]:
def create_asym_yaml(project_dir="",project_name=""):
    path=f"/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/{project_name}/volatile/asym"
    data = {}
    nested_dict = {}
    current_dict = nested_dict
    L=0
    Lmax=100
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            subdir = os.path.join(root,dir)
            split_subdir = subdir.split("/")
            asym_index = split_subdir.index("asym")
            result = split_subdir[asym_index+1:]
            if any(element.startswith("out") for element in result) and \
               result.index(next(filter(lambda x: x.startswith("out"), result))) < len(result) - 1 and \
                ".ipynb_checkpoints" not in result:

                string = ""
                # Now get sPlot or Sdbnd data
                if("outObsBins_splot" in result):
                    x_values, param_names, param_values, param_errors = read_splot(subdir)
                    result[-2]="splot"
                elif("outObsBins_splot_sig" in result):
                    x_values, param_names, param_values, param_errors = read_splot(subdir)
                    result[-2]="splot_sig"
                elif("outObsBins_splot_bg" in result):
                    x_values, param_names, param_values, param_errors = read_splot(subdir)
                    result[-2]="splot_bg"
                elif("outSdbndBins" in result):
                    x_values, param_names, param_values, param_errors = read_sideband(subdir)
                    result[-2]="sideband"
                elif("outObsBins" in result):
                    x_values, param_names, param_values, param_errors = read_splot(subdir)
                    result[-2]="standard"
                else:
                    continue
                data_list=result
                
                if(x_values==None):
                    continue
                
                for i, key in enumerate(data_list[:-1]):
                    if(not key in current_dict):  
                        current_dict[key] = {}
                    if(i==len(data_list[:-1])-1): break
                    current_dict = current_dict[key]

                
                if 'x' in current_dict[key]:
                    current_dict[key]["x"].extend(x_values)
                    for p, v, e in zip(param_names, param_values, param_errors):
                        if p in current_dict[key]:
                            current_dict[key][p]["value"].extend(v)
                            current_dict[key][p]["error"].extend(e)
                        else:
                            current_dict[key][p] = {"value": v, "error": e}
                else:
                    current_dict[key] = {"x": x_values}
                    for p, v, e in zip(param_names, param_values, param_errors):
                        current_dict[key][p] = {"value": v, "error": e}
                current_dict = nested_dict
                
    with open(f"{project_dir}/{project_name}/dihadron_binning.yaml", "w") as file:
        yaml.dump(nested_dict, file)

In [ ]:
def get_asym_yaml(project_dir,project_name):
    # Load YAML data from file
    with open(f"{project_dir}/{project_name}/dihadron_binning.yaml", "r") as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data

In [1]:
def get_data_from_yaml(data, headers, par):
    current_dict = data
    for header in headers:
        current_dict = current_dict[header]
    
    x = current_dict["x"]
    y = current_dict[par]["value"]
    yerr = current_dict[par]["error"]
    
    # Sort the x, y, and yerr arrays by x
    sorted_indices = sorted(range(len(x)), key=lambda i: x[i])
    x = [x[i] for i in sorted_indices]
    y = [y[i] for i in sorted_indices]
    yerr = [yerr[i] for i in sorted_indices]

    return x, y, yerr

In [ ]:
def print_dict_keys(dict_obj, level=0):
    # Print keys of dictionary at current level
    key0=""
    for key in dict_obj.keys():
        print("  " * level + key)
        if(key0==""):
            key0=key
    # If value is another dictionary, recursively print its keys
    if isinstance(dict_obj[key0], dict):
        print_dict_keys(dict_obj[key0], level+1)

In [5]:
# Get the injection functions for the specific plot
def get_inject_func(binyaml,plot):
    binningStructures = None

    with open(binyaml) as f:
        binningStructures = yaml.safe_load(f)

    sigfunc_dict = {}
    bgfunc_dict = {}

    for structure in binningStructures['binningStructures']:
        name = str(structure['name'])
        sigfuncs = structure.get('inject_sigfuncs',[])
        bgfuncs = structure.get('inject_bgfuncs',[])

        if(name==plot):
            return sigfuncs,bgfuncs
    

def get_inject_plot(func="",xmin=0,xmax=1,y=0): # y is reserved for 2d binning
    x = np.linspace(xmin,xmax,100)
    z = eval(func.replace("sin","np.sin"))
    return np.array(x),np.array(z)

In [4]:
def get_modulations(L):
    char_vec = []
    str_vec = []
    base_str_vec=[]
    for l in range(0, L+1):
        for m in range(1, l+1):
            base_str_vec.append(f"\sin({m}\phi_h-{m}\phi_R)")
            if(m==1):
                string = "\sin(" + "\phi_{h}-" + "\phi_{R})"
            else:
                string = "\sin(" + str(m) +"\phi_{h}-" + str(m) +"\phi_{R})"
            str_vec.append(string)
        for m in range(-l, l+1):
            base_str_vec.append(f"\sin({1-m}\phi_h+{m}\phi_R)")
            if(m==1):
                string = "\sin("+"\phi_{R})"
            elif(m==2):
                string = "\sin("  + "-\phi_{h}+" + str(m) +"\phi_{R})"
            elif(m==0):
                string = "\sin("+"\phi_{h})"
            elif(m==-1):
                string = "\sin(" + str(1-m) + "\phi_{h}-" + "\phi_{R})"
            elif(m<0):
                string = "\sin(" + str(1-m) + "\phi_{h}" + str(m) +"\phi_{R})"
            else:
                string = "\sin(" + str(1-m) + "\phi_{h}+" + str(m) +"\phi_{R})"
            str_vec.append(string)
    # Remove duplicate entries
    idx=np.argsort(base_str_vec)
    
    str_vec=list(np.array(str_vec)[idx])
    str_vec = list(dict.fromkeys(str_vec))
    cidx = 0
    for c in range(ord('A'), ord('A')+len(str_vec)):
        string = ""
        string += chr(c)
        char_vec.append(string)
        cidx += 1
    return (char_vec, str_vec)

# Bin Migration helper code

In [2]:
import yaml
import uproot
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import csv
from itertools import product
import ROOT
import shutil

Welcome to JupyROOT 6.22/06


In [3]:
def save_confusion_matrix(confusion_matrix, file_name):
    with open(file_name, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for row in confusion_matrix:
            csv_writer.writerow(row)

In [22]:
def get_cuts(channel="",cut_name="",out_dir=""):
    
    cut_library_file = "/work/clas12/users/gmat/clas12/clas12_dihadrons/utils/cut_library.yaml"
    if(out_dir!=""):
        shutil.copy(cut_library_file, out_dir)
    
    with open(cut_library_file) as f:
        cuts = yaml.safe_load(f)

    cut_var = []
    cut_min = []
    cut_max = []
    cut_strings = []
    if(cut_name!=""):
        cut_dict = cuts.get(cut_name, {})  # get the dictionary of cuts for the given cut_name
        for var, cut_range in cut_dict.items():
            cut_var.append(var)
            cut_min.append(float(cut_range.split(" ")[0]))
            cut_max.append(float(cut_range.split(" ")[1]))
            cut_strings.append(f"{cut_min[-1]} < {cut_var[-1]} < {cut_max[-1]}")
    if("pi0_pi0" in channel):
        cut_var.append("p_11")
        cut_var.append("p_12")
        cut_var.append("p_21")
        cut_var.append("p_22")
        cut_min.append(0.9)
        cut_min.append(0.9)
        cut_min.append(0.9)
        cut_min.append(0.9)
        cut_max.append(1.0)
        cut_max.append(1.0)
        cut_max.append(1.0)
        cut_max.append(1.0)
        cut_strings.append("p(gamma1)>0.9")
        cut_strings.append("p(gamma2)>0.9")
        cut_strings.append("p(gamma3)>0.9")
        cut_strings.append("p(gamma4)>0.9")
            
    elif(channel=="piplus_pi0" or channel=="piminus_pi0"):
        cut_var.append("p_21")
        cut_var.append("p_22")
        cut_min.append(0.9)
        cut_min.append(0.9)
        cut_max.append(1.0)
        cut_max.append(1.0)
        cut_strings.append("p(gamma1)>0.9")
        cut_strings.append("p(gamma2)>0.9")
        
    return cut_var,cut_min,cut_max,cut_strings

'0.0 < xF1 < 1.0&&0.0 < xF2 < 1.0&&0.0 < z < 0.95&&p(gamma1)>0.9&&p(gamma2)>0.9'

In [7]:
def bin_migration(project_dir, channel, version, binstruct, cut_name):
    file_path = f"{project_dir}/volatile/data/{channel}/{version}_merged.root"
    
    dimension_names = binstruct["dimensionNames"]
    bin_edges = binstruct["binEdges"]
    
    dimension_names_underscore = '_'.join(dimension_names)
    
    out_dir = f"{project_dir}/systematics/bin_migration/{channel}"
    # Check if the directory exists
    if not os.path.exists(out_dir):
        # If it does not exist, create it using os.makedirs
        os.makedirs(out_dir)
    
    cut_var,cut_min,cut_max,cut_strings = get_cuts(channel,cut_name,out_dir)
        
    with uproot.open(file_path) as file:
        ttree = file["dihadron"]
        
        # apply the cuts to select events
        all_cuts = np.array(ttree["MCmatch"].array()==1)
        for v,vmin,vmax in zip(cut_var,cut_min,cut_max):
            varr = np.array(ttree[v].array())
            all_cuts*= (varr>vmin) * (varr<vmax)

        reco_branches = [ttree[dimension_name].array()[all_cuts] for dimension_name in dimension_names]
        true_branches = [ttree[f"true{dimension_name}"].array()[all_cuts] for dimension_name in dimension_names]
    
    n_dimensions = len(dimension_names)
    digitize = lambda values, edges: np.digitize(values, edges) - 1
    reco_bins = [digitize(reco, edges) for reco, edges in zip(reco_branches, bin_edges)]
    true_bins = [digitize(true_, edges) for true_, edges in zip(true_branches, bin_edges)]

    all_bins = list(product(*[range(len(edges) - 1) for edges in bin_edges]))
    bin_labels = [tuple(map(str, bins)) for bins in all_bins]

    confusion_matrix = np.zeros((len(all_bins), len(all_bins)))
    for reco, true_ in zip(zip(*reco_bins), zip(*true_bins)):
        if all(0 <= reco_bin < len(edges) - 1 for reco_bin, edges in zip(reco, bin_edges)) and all(0 <= true_bin < len(edges) - 1 for true_bin, edges in zip(true_, bin_edges)):
            reco_idx = all_bins.index(reco)
            true_idx = all_bins.index(true_)
            confusion_matrix[reco_idx, true_idx] += 1
    
    # Set bins with 0 sum to 0
    col_sums = confusion_matrix.sum(axis=0)
    col_sums[col_sums==0]=1

    # Normalize the confusion matrix
    cm = confusion_matrix / col_sums
    #cm = confusion_matrix / confusion_matrix.sum(axis=0, keepdims=True)
    cm = np.flipud(cm)
    
    # Calculate the size of the figure based on the number of bins in each dimension
    fig_size = (cm.shape[0], 0.75*cm.shape[0])

    norm = mcolors.LogNorm(vmin=cm.min()+1e-6, vmax=cm.max())
    fig, ax = plt.subplots(figsize=fig_size)
    im = plt.imshow(cm, interpolation="nearest", cmap="Blues", aspect="auto", norm=norm)

    ax.set_xticks(np.arange(len(all_bins)))
    ax.set_yticks(np.arange(len(all_bins)))
    # Convert tuple of integers to list of integers as tick labels
    ax.set_xticklabels(list(map(lambda x: str(x).replace("(", "").replace(",)", "").replace(")","").replace("'", ""), bin_labels)), fontsize=12)
    ax.set_yticklabels(list(map(lambda x: str(x).replace("(", "").replace(",)", "").replace(")","").replace("'", ""), bin_labels[::-1])), fontsize=12)
    # Add x-axis and y-axis labels
    ax.set_xlabel("Monte Carlo Generated Bin", fontsize=14)
    ax.set_ylabel("Reconstructed Bin", fontsize=14)
    # Add title
    ax.set_title(f"Bin Migration for {dimension_names} bins",fontsize=22)
    
    for i in range(len(all_bins)):
        for j in range(len(all_bins)):
            ax.text(j, i, f"{cm[i, j]:.3f}",
                    ha="center", va="center", color="black" if cm[i, j] < 0.5 * cm.max() else "white", fontsize=12)

    bin_centers = [0.5 * (np.array(edges[:-1]) + np.array(edges[1:])) for edges in bin_edges]
    legend_elements = [f"{dim}: bin {i} = {center:.2f}" for dim, centers in zip(dimension_names, bin_centers) for i, center in enumerate(centers)]
    legend_text = "\n".join(legend_elements)
    legend_text = "Cuts:\n"+"\n".join(cut_strings)+"\n\n"+legend_text
    ax.annotate(legend_text, xy=(1.05, 0.5), xycoords="axes fraction", fontsize=15, ha="left", va="center")

    plt.tight_layout()
    plotname = f'{out_dir}/figure_{dimension_names_underscore}_{version}_{cut_name}.png'
    plt.savefig(plotname)
    # Save the confusion matrix
    csvname = f'{out_dir}/matrix_{dimension_names_underscore}_{version}_{cut_name}.csv'
    save_confusion_matrix(cm, csvname)
    print("Saved to",csvname)
    plt.close()

# Kinematics Helper Code

In [3]:
def kinematics_binning(project_dir, channel, version, binstruct, branchname, cut_name, xbins=100,xmin=0,xmax=1,Nmax=1e5):
    Nmax=int(Nmax)
    file_path = f"{project_dir}/volatile/data/{channel}/{version}_merged.root"
    dimension_names = binstruct["dimensionNames"]
    bin_edges = binstruct["binEdges"]
    
    # Open the ROOT file and TTree
    root_file = ROOT.TFile(file_path, "READ")
    tree = root_file.Get("dihadron")

    # Determine if the binning is 1D or 2D
    is_2d_binning = len(dimension_names) == 2

    # Define padding for major axes
    x_padding = 0.12 if is_2d_binning else 0.05
    y_padding = 0.2
    # Create the grid of TPads
    n_columns = len(bin_edges[0]) - 1
    n_rows = len(bin_edges[1]) - 1 if is_2d_binning else 1
    pad_width = (1.0 - 2*x_padding) / n_columns 
    pad_height = (1.0 - 2*y_padding) / n_rows 
    
    pads = []

    # Create the TCanvas
    canvas = ROOT.TCanvas("canvas", "Kinematics", 100+200*n_columns, 200+130*n_rows)
    canvas.cd()

    for i in range(n_rows):
        pad_row = []
        for j in range(n_columns):
            pad_name = f"pad_{i}_{j}"
            pad = ROOT.TPad(pad_name, pad_name, j * pad_width+x_padding, (n_rows - i - 1) * pad_height+y_padding, (j + 1) * pad_width+x_padding, (n_rows - i) * pad_height+y_padding)
            pad.SetMargin(0, 0, 0, 0)
            if i == 0 and j == 0:
                pad.Draw()
            else:
                pad.Draw("same")
            pad_row.append(pad)
        pads.append(pad_row)

    hists = []
    max_height = 0
    # Draw histograms in TPads
    for i in range(n_rows):
        for j in range(n_columns):
            pads[i][j].cd()
            ROOT.gStyle.SetOptStat(0)
            x_bin_range = f"{dimension_names[0]} > {bin_edges[0][j]} && {dimension_names[0]} < {bin_edges[0][j+1]}"
            y_bin_range = f"{dimension_names[1]} > {bin_edges[1][i]} && {dimension_names[1]} < {bin_edges[1][i+1]}" if is_2d_binning else ""
            cut = f"{x_bin_range}"
            if(is_2d_binning):
                cut+=f"&&{y_bin_range}"
            
            hist = ROOT.TH1F(f"hist_{i}_{j}", "", xbins, xmin, xmax)
            tree.Draw(f"{branchname}>>hist_{i}_{j}", cut, "goff",Nmax)
            hist.Draw()

            hists.append(hist)
            if hist.GetMaximum() > max_height:
                max_height = hist.GetMaximum()

    # Set the maximum y-axis range of all histograms to max_height
    for hist in hists:
        hist.SetMaximum(max_height)
    
    
    # Create a TGaxis for the x-axis counts
    external_x_axis_bottom_left = ROOT.TGaxis(x_padding, y_padding, x_padding+pad_width, y_padding, xmin, xmax, 508, "S")
    external_x_axis_bottom_left.SetTickLength(0.01)
    external_x_axis_bottom_left.SetTitle(branchname)
    external_x_axis_bottom_left.SetTitleOffset(0.5)
    external_x_axis_bottom_left.SetTitleSize(0.05)
    external_x_axis_bottom_left.SetLabelSize(0.04)
    external_x_axis_bottom_left.SetLabelOffset(0.005)
    external_x_axis_bottom_left.SetLabelFont(42)

    # Plot the y-axis counts in the bottom left corner
    axis_min = 0
    axis_max = hist.GetMaximum()
    axis_range = axis_max - axis_min
    hist.GetYaxis().SetTickLength(0)
    external_y_axis_left_bottom = ROOT.TGaxis(x_padding, y_padding, x_padding, y_padding+pad_height, axis_min, axis_max, 505, "S")
    external_y_axis_left_bottom.SetTickLength(0.01) # set the tick size to 0.02
    external_y_axis_left_bottom.SetTitle("" if is_2d_binning else "Counts")
    external_y_axis_left_bottom.SetTitleOffset(0.3)
    external_y_axis_left_bottom.SetTitleSize(0.08)
    external_y_axis_left_bottom.SetLabelSize(0.03 if is_2d_binning else 0.07)
    external_y_axis_left_bottom.SetLabelOffset(0.005)
    external_y_axis_left_bottom.SetLabelFont(42)
    canvas.cd()
    external_x_axis_bottom_left.Draw()
    external_y_axis_left_bottom.Draw()
    # X-axis
    x_axis_n_div = len(bin_edges[0]) - 1
    x_axis_labels = [f"{np.round(bin_edges[0][i],2)}" for i in range(x_axis_n_div + 1)]
    external_x_axis = ROOT.TGaxis(x_padding, y_padding/1.8, 1-x_padding, y_padding/1.8, 0, x_axis_n_div, x_axis_n_div, "<")
    #external_x_axis.SetTitle(dimension_names[0])
    latex=ROOT.TLatex()
    latex.SetTextSize(0.08)
    latex.DrawLatexNDC(1-0.8*x_padding,0.5*y_padding,dimension_names[0])
    external_x_axis.SetLabelSize(0.05 if is_2d_binning else 0.07)
    external_x_axis.SetTitleSize(0.07 if is_2d_binning else 0.1)
    external_x_axis.SetTitleOffset(0.72)
    external_x_axis.SetLabelFont(42)
    for i in range(x_axis_n_div + 1):
        external_x_axis.ChangeLabel(i + 1, -1, -1, -1, -1, -1, x_axis_labels[i])
    #external_x_axis.CenterTitle()
    external_x_axis.Draw("same")

    if is_2d_binning:
        y_axis_n_div = len(bin_edges[1]) - 1
        y_axis_labels = [f"{np.round(bin_edges[1][i],2)}" for i in range(y_axis_n_div + 1)]
        external_y_axis = ROOT.TGaxis(x_padding/1.5, y_padding, x_padding/1.5, 1-y_padding, 0, y_axis_n_div, y_axis_n_div, "S")
        external_y_axis.SetTitle(dimension_names[1])
        external_y_axis.SetTitleOffset(0.4)
        external_y_axis.SetTickLength(0.013)
        external_y_axis.SetLabelSize(0.04)
        external_y_axis.SetTitleSize(0.07)
        external_y_axis.SetLabelFont(42)
        external_y_axis.CenterTitle()
        for i in range(y_axis_n_div + 1):
            external_y_axis.ChangeLabel(i + 1, -1, -1, -1, -1, -1, y_axis_labels[i])
        external_y_axis.Draw()
    
    # Create a TPaveText object for the title
    title = ROOT.TPaveText(0.1, 1-0.9*y_padding, 0.9, 0.98, "NDC")
    title.SetFillColor(0)
    title.SetBorderSize(0)
    title.SetTextFont(42)
    title.SetTextAlign(22)
    title.SetTextSize(0.1)
    CHANNEL=channel.replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")
    VERSION=version.replace("_"," ")
    
    title.AddText(f"{branchname} distributions for {CHANNEL}, {VERSION}")
    # Draw the TPaveText object on the canvas
    title.Draw()
    canvas.Draw()
    canvas.SaveAs("output_canvas.png")
    
    # Show what the binning is
    kinematics_show_binning(file_path,"dihadron",dimension_names,bin_edges,is_2d_binning,Nmax)
    
    # Cleanup
    root_file.Close()

In [10]:
def kinematics_show_binning(root_file_path, tree_name, dimension_names, bin_edges, is_2d_binning,Nmax):
    # Open the ROOT file and TTree
    root_file = ROOT.TFile(root_file_path, "READ")
    tree = root_file.Get(tree_name)

    # Create the TCanvas
    canvas = ROOT.TCanvas("canvas", "Histogram", 800, 600)

    # Draw the histogram
    if is_2d_binning:
        hist = ROOT.TH2F("hist", "", 100, bin_edges[0][0], bin_edges[0][-1], 100,bin_edges[1][0], bin_edges[1][-1])
        tree.Draw(f"{dimension_names[1]}:{dimension_names[0]}>>hist", "", "colz",Nmax)
        hist.Draw("colz")
    else:
        hist = ROOT.TH1F("hist", "", 100, bin_edges[0][0], bin_edges[0][-1])
        tree.Draw(f"{dimension_names[0]}>>hist", "", "",Nmax)
        hist.Draw()

    # Add TLines to demarcate binning
    lines=[]
    if is_2d_binning:
        for i in range(len(bin_edges[0])-1):
            lines.append(ROOT.TLine(bin_edges[0][i], bin_edges[1][0], bin_edges[0][i], bin_edges[1][-1]))
            lines[-1].SetLineStyle(1)
            lines[-1].Draw()
        for i in range(len(bin_edges[1])-1):
            lines.append(ROOT.TLine(bin_edges[0][0], bin_edges[1][i], bin_edges[0][-1], bin_edges[1][i]))
            lines[-1].SetLineStyle(1)
            lines[-1].Draw()
    else:
        for i in range(len(bin_edges[0])-1):
            lines.append(ROOT.TLine(bin_edges[0][i], 0, bin_edges[0][i], hist.GetMaximum()))
            lines[-1].SetLineStyle(1)
            lines[-1].Draw("same")

    # Set axis labels
    if is_2d_binning:
        hist.GetXaxis().SetTitle(dimension_names[0])
        hist.GetYaxis().SetTitle(dimension_names[1])
    else:
        hist.GetXaxis().SetTitle(dimension_names[0])
        hist.GetYaxis().SetTitle("Counts")

    # Update the canvas
    canvas.Update()
    
    canvas.SaveAs("output_binning_canvas.png")

    # Cleanup
    root_file.Close()

# Simple plotter helper code

In [1]:
#This function takes in a histogram object and a TChain object with the EventTree already loaded. 
#It also takes in the name of a branch for binning (either 1D or 2D) and optionally the name of a branch for scaling. 
#If the histogram is 2D, it takes in another name of a branch for binning. 
#The function then fills the histogram and a corresponding "counts" histogram using the TChain and the binning branches. 
#The counts histogram is used to normalize the original histogram, avoiding dividing by zero errors in empty bins. 
#The normalization is done by dividing the content of each bin in the original histogram by the corresponding bin content in the counts histogram.
#If the user specified a scale_branch, the histogram is filled using the product of the scale_branch and the cut string.
def fill_histogram(hist, chain, bin_branch_x, bin_branch_y=None, scale_branch=None, cut=""):
    scale_cut=scale_branch+("*("+cut+")" if cut else "")
    if bin_branch_y is None:
        # 1-dimensional histogram
        counts_hist = ROOT.TH1F("counts_hist", "", hist.GetNbinsX(), hist.GetXaxis().GetBinLowEdge(1), hist.GetXaxis().GetBinUpEdge(hist.GetNbinsX()))
        # Fill the histograms
        chain.Draw(f"{bin_branch_x}>>hist", scale_cut)
        chain.Draw(f"{bin_branch_x}>>counts_hist",cut)
        print(counts_hist.GetEntries())
        # Normalize the histogram
        for i in range(1, hist.GetNbinsX()+1):
            bin_content = hist.GetBinContent(i)
            bin_error = hist.GetBinError(i)
            count = counts_hist.GetBinContent(i)
            if count != 0:
                hist.SetBinContent(i, bin_content/count)
                hist.SetBinError(i,bin_error/count)
    
    else:
        # 2-dimensional histogram
        counts_hist = ROOT.TH2F("counts_hist", "", hist.GetNbinsX(), hist.GetXaxis().GetBinLowEdge(1), hist.GetXaxis().GetBinUpEdge(hist.GetNbinsX()), hist.GetNbinsY(), hist.GetYaxis().GetBinLowEdge(1), hist.GetYaxis().GetBinUpEdge(hist.GetNbinsY()))
        # Fill the histograms
        chain.Draw(f"{bin_branch_y}:{bin_branch_x}>>hist", scale_cut)
        chain.Draw(f"{bin_branch_y}:{bin_branch_x}>>counts_hist",cut)
        
        # Normalize the histogram
        for i in range(1, hist.GetNbinsX()+1):
            for j in range(1, hist.GetNbinsY()+1):
                bin_content = hist.GetBinContent(i, j)
                bin_error = hist.GetBinError(i,j)
                count = counts_hist.GetBinContent(i, j)
                if count < 100:
                    hist.SetBinContent(i, j, 0)
                    hist.SetBinError(i,0)
                    continue
                if count != 0:
                    hist.SetBinContent(i, j, bin_content/count)
                    hist.SetBinError(i,bin_error/count)
    
    return hist

In [2]:
def format_variable_name(variable_name):
    # Remove "true" from the variable name using regular expressions
    variable_name = re.sub(r'^true', '', variable_name)
    # Add "$" and "_{true}$"
    variable_name = f"${variable_name}_{{true}}$"
    return variable_name

In [ ]:
def plot_graph(project_dir, channel, version, ttree,
               x_col, x_min, x_max, x_bins = None, x_title = None,
               y_col=None, y_min=None, y_max=None, y_bins=None, set_log_y = False, y_title = None,
               z_min=None, z_max=None, z_col = None, set_log_z = False, z_title=None,
               cut="", nbins=50, latex_xmax=0.6, latex_ymax=0.3,
               alias_x="_X_", alias_y="_Y_", alias_z="_Z_",
               suffix=""):
    c=ROOT.TCanvas("","",500,500)
    
    file_path = f"{project_dir}/volatile/data/{channel}/{version}_merged.root"
    
    # Open the ROOT file and TTree
    root_file = ROOT.TFile(file_path, "READ")
    
    chain = root_file.Get(ttree)
    print(chain)
    # Create aliases for the x and y branches
    chain.SetAlias(alias_x, x_col)
    if y_col is not None:
        chain.SetAlias(alias_y, y_col)
    if z_col is not None:
        chain.SetAlias(alias_z, z_col)

    if y_col is None:
        # 1-dimensional histogram
        # Create a TH1F histogram object
        hist = ROOT.TH1F("hist", "", (x_bins if x_bins else nbins), x_min, x_max)
        
        # Set the x axis title
        hist.GetXaxis().SetTitle(x_col if x_title==None else x_title)
        hist.GetYaxis().SetTitle(y_col if y_title==None else y_title)
        hist.GetXaxis().SetTitleSize(0.055)
        hist.GetYaxis().SetTitleSize(0.055)
        # Fill the histogram with data from the TChain
        if(z_col!=None):
            fill_histogram(hist, chain, bin_branch_x=x_col, bin_branch_y=None, scale_branch=alias_z, cut=cut)
            if(y_title==None):
                hist.GetYaxis().SetTitle(f"Average {z_col}")
        else:
            chain.Draw(f"{alias_x}>>hist", cut)
        
        # Draw the histogram
        hist.Draw()
        
        title = f"Histogram of {x_col} with {cut}"
    
    else:
        # 2-dimensional histogram
        # Create a TH2F histogram object
        hist = ROOT.TH2F("hist", "", (x_bins if x_bins else nbins), x_min, x_max, (y_bins if y_bins else nbins), y_min, y_max)
        
        # Set the x and y axis titles
        hist.GetXaxis().SetTitle(x_col if x_title==None else x_title)
        hist.GetXaxis().SetTitleSize(0.07)
        hist.GetXaxis().SetTitleOffset(0.7)
        hist.GetYaxis().SetTitle(y_col if y_title==None else y_title)
        hist.GetYaxis().SetTitleSize(0.07)
        hist.GetYaxis().SetTitleOffset(0.7)
        # Fill the histogram with data from the TChain
        if(z_col!=None):
            fill_histogram(hist, chain, bin_branch_x=x_col, bin_branch_y=y_col, scale_branch=alias_z, cut=cut)
        else:
            chain.Draw(f"{alias_y}:{alias_x}>>hist", cut)
        
        # Draw the histogram
        hist.Draw("COLZ")
        
        title = f"Heatmap of {x_col} vs. {y_col} with {cut}"
        pad = ROOT.gPad
        pad.Modified()
        pad.Update()
        palette = hist.GetListOfFunctions().FindObject("palette")
        palette.SetX1NDC(1-0.13)
        palette.SetX2NDC(1-0.08)
        palette.SetY1NDC(0.14)
        #pad.Modified()
        #pad.Update()
        hist.SetContour(99) # set the number of levels
        if(z_min!=None and z_max!=None):
            hist.GetZaxis().SetRangeUser(z_min,z_max)
            hist.GetZaxis().SetNdivisions(510)
        if(z_col!=None):
            if(z_title!=None):
                hist.GetZaxis().SetTitle(z_title)
            else:
                hist.GetZaxis().SetTitle(f"Average {z_col}")
            hist.GetZaxis().SetTitleOffset(1.2)
        if(set_log_z):
            c.SetLogz(1)
        if(set_log_y):
            c.SetLogy(1)
        
    
    ROOT.gStyle.SetOptStat(0)
    ROOT.gPad.SetLeftMargin(0.14)
    ROOT.gPad.SetBottomMargin(0.14)
    ROOT.gPad.SetRightMargin(0.14)
    ROOT.gStyle.SetPalette(ROOT.kRainBow)
    hist.Draw("COLZ")
    
    # Create a TLatex object for the title
    latex = ROOT.TLatex()
    latex.SetTextFont(42)
    latex.SetTextSize(0.04)
    dy=0.04
    CHANNEL=channel.replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")
    VERSION=version.replace("RGA","RG-A").replace("RGC","RG-C").replace("_"," ")
    latex.DrawLatexNDC(latex_xmax,latex_ymax,CHANNEL+" " + VERSION)
    
    #canvas.SaveAs(f"fidplots/plot_{x_col}_{y_col}_{suffix}.png")
    return copy.deepcopy(c)
    #return copy.deepcopy(hist), title

# Dihadron Counts Helper Code

In [2]:
def gen_Mgg_dists(project_name, version, Nmax=10000,ML=True):
    path_to_repo="/work/clas12/users/gmat/clas12/clas12_dihadrons"
    project_dir=f"{path_to_repo}/projects/{project_name}"

    # Define the list of ROOT files to open
    channels = ["piplus_piminus", "piplus_pi0", "piplus_piplus","pi0_pi0","piminus_pi0", "piminus_piminus"]
    file_list = [f"{project_dir}/volatile/data/{channel}/{version}_merged.root" for channel in channels if "pi0" in channel]
    for file in file_list:
        if("pi0_pi0" in file):
            file_list.append(file)
            file_list=sorted(file_list)[::-1]
            break

    channels = ["piplus_pi0","piminus_pi0","pi0_pi0 1st","pi0_pi0 2nd"]
    cut_names = ["v4","v4","v1","v1"]
    # Create a new TCanvas to hold the histograms
    canvas = ROOT.TCanvas("canvas", "Mh Distribution", 1000, 600)
    left_pad = ROOT.TPad("left_pad", "Left Pad", 0, 0, 0.7, 1)
    left_pad.Draw()
    left_pad.cd()
    ROOT.gPad.SetLeftMargin(0.13)
    ROOT.gStyle.SetOptStat(0)
    # Create a new TLegend to hold the histogram labels
    legend = ROOT.TLegend(0.7, 0.5, 0.9, 0.9)

    hists=[]
    colors=[2,9,8,30]
    cstrs=[]
    CHANNELS=[]

    # Loop over each ROOT file in the list
    for i, file_name in tqdm(enumerate(file_list)):
        # Open the ROOT file and extract the "dihadron" TTree
        file = ROOT.TFile(file_name)
        tree = file.Get("dihadron")

        # Create a new TH1F histogram for the "Mh" distribution
        hist_name = "hist_{}".format(i)
        hist_title = "Diphoton Mass Distribution;M_{#gamma#gamma}[GeV];Counts"
        hist = ROOT.TH1F(hist_name, hist_title, 100, 0, 0.4)

        # If ML, apply cut
        cut=""
        cut_var,cut_min,cut_max,cut_strings = get_cuts(channels[i],cut_names[i])
        if(not ML):
            iML=[not "p_" in cv for cv in cut_var]
            cut_var=np.array(cut_var)[iML]
            cut_min=np.array(cut_min)[iML]
            cut_max=np.array(cut_max)[iML]
            cut_strings=np.array(cut_strings)[iML]
            if("pi0" in channels[i]):
                # Append the new values to the numpy arrays
                cut_var = np.append(cut_var, "isGoodEventWithoutML")
                cut_min = np.append(cut_min, 0.9)
                cut_max = np.append(cut_max, 1.1)
                cut_strings = np.append(cut_strings, "Traditional Cuts")
                
        cut="&&".join([f"{cv}>{cl}&&{cv}<{ch}" for cv,cl,ch in zip(cut_var,cut_min,cut_max)])
        cstrs.append(cut_strings)
        # Fill the histogram with the "Mh" values from the TTree
        tree.Draw("{}>>{}".format("M1" if "pi0_pi0 1st" in channels[i] else "M2",hist_name),cut,"goff",Nmax)
        # Set the histogram line color and add it to the TCanvas
        hist.SetLineColor(colors[i])
        hist.SetLineWidth(2)
        CHANNEL=channels[i].replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")

        hists.append(copy.deepcopy(hist))
        hists[-1].Draw("same" if i!=0 else "")
        # Add the histogram label to the TLegend
        legend.AddEntry(hists[-1], CHANNEL, "l")

    ymax=0
    for h in hists:
        if(h.GetMaximum()>ymax):
            ymax=h.GetMaximum()
    hists[0].GetYaxis().SetRangeUser(0,ymax*1.2)
    # Set the x and y axis labels
    canvas.SetTitle("Mh Distribution")

    # Draw the TLegend on the TCanvas
    legend.Draw()

    # Create a TLatex object and set the font and size
    latex = ROOT.TLatex()
    latex.SetTextFont(42)
    latex.SetTextSize(0.04)

    # Add a label to the TCanvas using TLatex drawn with NDC
    label_text = version.replace("_"," ")+" Sample"
    label_x = 0.15
    label_y = 0.85
    latex.DrawLatexNDC(label_x, label_y, label_text)


    canvas.cd()
    # Get the right TPad
    right_pad = ROOT.TPad("right_pad", "Right Pad", 0.65, 0, 1, 1)
    right_pad.SetLeftMargin(0)
    right_pad.Draw()
    right_pad.cd()

    # Divide the info pad into a 2x3 grid of text boxes
    right_pad.Divide(2, 3)
    textboxes=[]
    # Add text boxes to the info pad
    for i in range(4):
        textbox = ROOT.TPaveText(0, 0, 1, 1, "NDC")
        textbox.SetTextAlign(22)
        textbox.SetTextSize(0.08)
        textbox.SetFillColor(0)
        textbox.SetBorderSize(0)
        right_pad.cd(i+1)
        if(i<4):
            CHANNEL=channels[i].replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")
            textbox.AddText("#scale[1.2]{"+"#color["+str(colors[i])+"]"+"{"+"{}".format(CHANNEL)+"} channel cuts}")
            for cstr in cstrs[i]:
                textbox.AddText(cstr)
        textboxes.append(copy.deepcopy(textbox))
        textboxes[-1].Draw()


    # Create two vertical lines
    line1 = ROOT.TLine(0.106, 0, 0.106,ymax)
    line2 = ROOT.TLine(0.164, 0, 0.164, ymax)

    # Set the line colors and styles
    line1.SetLineColor(1)
    line2.SetLineColor(1)
    line1.SetLineWidth(2)
    line2.SetLineWidth(2)
    line1.SetLineStyle(1)
    line2.SetLineStyle(1)

    left_pad.cd()
    # Draw the lines on the canvas
    line1.Draw("same")
    line2.Draw("same")

    # Update the TCanvas to show the histograms
    canvas.Draw()
    return copy.deepcopy(canvas)

In [1]:
def gen_Mh_dists(project_name, channels, cut_names, version, Nmax=10000,ML=True,log_y=False):
    path_to_repo="/work/clas12/users/gmat/clas12/clas12_dihadrons"
    project_dir=f"{path_to_repo}/projects/{project_name}"
    
    # Define the list of ROOT files to open
    file_list = [f"{project_dir}/volatile/data/{channel}/{version}_merged.root" for channel in channels]
    
    # Create a new TCanvas to hold the histograms
    canvas = ROOT.TCanvas("canvas", "Mh Distribution", 1000, 600)
    left_pad = ROOT.TPad("left_pad", "Left Pad", 0, 0, 0.7, 1)
    left_pad.Draw()
    left_pad.SetLogy(log_y)
    left_pad.cd()
    ROOT.gPad.SetLeftMargin(0.13)
    ROOT.gStyle.SetOptStat(0)
    # Create a new TLegend to hold the histogram labels
    legend = ROOT.TLegend(0.7, 0.5, 0.9, 0.9)

    hists=[]
    colors=[1,2,42,8,9,28]
    cstrs=[]

    # Loop over each ROOT file in the list
    for i, file_name in tqdm(enumerate(file_list)):
        # Open the ROOT file and extract the "dihadron" TTree
        file = ROOT.TFile(file_name)
        tree = file.Get("dihadron")

        # Create a new TH1F histogram for the "Mh" distribution
        hist_name = "hist_{}".format(i)
        hist_title = "Dihadron Mass Distribution;M_{#pi#pi}[GeV];Counts"
        hist = ROOT.TH1F(hist_name, hist_title, 100, 0.1, 1.6)

        # If ML, apply cut
        cut=""
        cut_var,cut_min,cut_max,cut_strings = get_cuts(channels[i],cut_names[i])
        if(not ML):
            iML=[not "p_" in cv for cv in cut_var]
            cut_var=np.array(cut_var)[iML]
            cut_min=np.array(cut_min)[iML]
            cut_max=np.array(cut_max)[iML]
            cut_strings=np.array(cut_strings)[iML]
            if("pi0" in channels[i]):
                # Append the new values to the numpy arrays
                cut_var = np.append(cut_var, "isGoodEventWithoutML")
                cut_min = np.append(cut_min, 0.9)
                cut_max = np.append(cut_max, 1.1)
                cut_strings = np.append(cut_strings, "Traditional Cuts")
        cut="&&".join([f"{cv}>{cl}&&{cv}<{ch}" for cv,cl,ch in zip(cut_var,cut_min,cut_max)])
        if("pi0" in channels[i]):
            cut+="&&M2>0.106&&M2<0.164"
            cut_strings = np.append(cut_strings,"0.106 < M_{#gamma#gamma} < 0.164")
        if("pi0_pi0" in channels[i]):
            cut+="&&M1>0.106&&M1<0.164"
        cstrs.append(cut_strings)
        # Fill the histogram with the "Mh" values from the TTree
        tree.Draw("Mh>>{}".format(hist_name),cut,"goff",Nmax)
        # Set the histogram line color and add it to the TCanvas
        hist.SetLineColor(colors[i])
        hist.SetLineWidth(2)
        CHANNEL=channels[i].replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")


        hists.append(copy.deepcopy(hist))
        hists[-1].Draw("same" if i!=0 else "")
        # Add the histogram label to the TLegend
        legend.AddEntry(hists[-1], CHANNEL, "l")

    ymax=0
    for h in hists:
        if(h.GetMaximum()>ymax):
            ymax=h.GetMaximum()
    hists[0].GetYaxis().SetRangeUser(1,ymax*1.2)
    # Set the x and y axis labels
    canvas.SetTitle("Mh Distribution")

    # Draw the TLegend on the TCanvas
    legend.Draw()

    # Create a TLatex object and set the font and size
    latex = ROOT.TLatex()
    latex.SetTextFont(42)
    latex.SetTextSize(0.04)

    # Add a label to the TCanvas using TLatex drawn with NDC
    label_text = version.replace("_"," ")+" Sample"
    label_x = 0.15
    label_y = 0.85
    latex.DrawLatexNDC(label_x, label_y, label_text)

    canvas.cd()
    # Get the right TPad
    right_pad = ROOT.TPad("right_pad", "Right Pad", 0.65, 0, 1, 1)
    right_pad.SetLeftMargin(0)
    right_pad.Draw()
    right_pad.cd()

    # Divide the info pad into a 2x3 grid of text boxes
    right_pad.Divide(2, 3)
    textboxes=[]
    # Add text boxes to the info pad
    for i in range(6):
        textbox = ROOT.TPaveText(0, 0, 1, 1, "NDC")
        textbox.SetTextAlign(22)
        textbox.SetTextSize(0.08)
        textbox.SetFillColor(0)
        textbox.SetBorderSize(0)
        right_pad.cd(i+1)
        if(i<6):
            CHANNEL=channels[i].replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")
            textbox.AddText("#scale[1.2]{"+"#color["+str(colors[i])+"]"+"{"+"{}".format(CHANNEL)+"} channel cuts}")
            for cstr in cstrs[i]:
                textbox.AddText(cstr)
        textboxes.append(copy.deepcopy(textbox))
        textboxes[-1].Draw()


    # Update the TCanvas to show the histograms
    canvas.Draw()
    return copy.deepcopy(canvas)

In [2]:
def gen_Mx_dists(project_name, channels, cut_names, version, Nmax=10000,ML=True,log_y=False):
    path_to_repo="/work/clas12/users/gmat/clas12/clas12_dihadrons"
    project_dir=f"{path_to_repo}/projects/{project_name}"
    
    # Define the list of ROOT files to open
    file_list = [f"{project_dir}/volatile/data/{channel}/{version}_merged.root" for channel in channels]
    
    # Create a new TCanvas to hold the histograms
    canvas = ROOT.TCanvas("canvas", "Missing Mass Distribution", 1000, 600)
    left_pad = ROOT.TPad("left_pad", "Left Pad", 0, 0, 0.7, 1)
    left_pad.SetLogy(log_y)
    left_pad.Draw()
    left_pad.cd()
    ROOT.gPad.SetLeftMargin(0.13)
    ROOT.gStyle.SetOptStat(0)
    # Create a new TLegend to hold the histogram labels
    legend = ROOT.TLegend(0.13, 0.9, 0.9, 0.98)
    legend.SetNColumns(6)
    hists=[]
    colors=[1,2,42,8,9,28]
    cstrs=[]

    # Loop over each ROOT file in the list
    for i, file_name in tqdm(enumerate(file_list)):
        # Open the ROOT file and extract the "dihadron" TTree
        file = ROOT.TFile(file_name)
        tree = file.Get("dihadron")

        # Create a new TH1F histogram for the "Mh" distribution
        hist_name = "hist_{}".format(i)
        hist_title = ";Missing Mass[GeV];Counts"
        hist = ROOT.TH1F(hist_name, hist_title, 100, 0, 4)

        # If ML, apply cut
        cut=""
        cut_var,cut_min,cut_max,cut_strings = get_cuts(channels[i],cut_names[i])
        if(not ML):
            iML=[not "p_" in cv for cv in cut_var]
            cut_var=np.array(cut_var)[iML]
            cut_min=np.array(cut_min)[iML]
            cut_max=np.array(cut_max)[iML]
            cut_strings=np.array(cut_strings)[iML]
            if("pi0" in channels[i]):
                # Append the new values to the numpy arrays
                cut_var = np.append(cut_var, "isGoodEventWithoutML")
                cut_min = np.append(cut_min, 0.9)
                cut_max = np.append(cut_max, 1.1)
                cut_strings = np.append(cut_strings, "Traditional Cuts")
        cut="&&".join([f"{cv}>{cl}&&{cv}<{ch}" for cv,cl,ch in zip(cut_var,cut_min,cut_max)])
        if("pi0" in channels[i]):
            cut+="&&M2>0.106&&M2<0.164"
            cut_strings = np.append(cut_strings,"0.106 < M_{#gamma#gamma} < 0.164")
        if("pi0_pi0" in channels[i]):
            cut+="&&M1>0.106&&M1<0.164"
        cstrs.append(cut_strings)
        # Fill the histogram with the "Mh" values from the TTree
        tree.Draw("Mx>>{}".format(hist_name),cut,"goff",Nmax)
        # Set the histogram line color and add it to the TCanvas
        hist.SetLineColor(colors[i])
        hist.SetLineWidth(2)
        CHANNEL=channels[i].replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")


        hists.append(copy.deepcopy(hist))
        hists[-1].Draw("same" if i!=0 else "")
        # Add the histogram label to the TLegend
        legend.AddEntry(hists[-1], CHANNEL, "l")

    ymax=0
    for h in hists:
        if(h.GetMaximum()>ymax):
            ymax=h.GetMaximum()
    hists[0].GetYaxis().SetRangeUser(1,ymax*(6 if log_y else 1.2))
    # Set the x and y axis labels
    canvas.SetTitle("Mx Distribution")

    # Draw the TLegend on the TCanvas
    legend.Draw()

    # Create a TLatex object and set the font and size
    latex = ROOT.TLatex()
    latex.SetTextFont(42)
    latex.SetTextSize(0.04)

    # Add a label to the TCanvas using TLatex drawn with NDC
    label_text = version.replace("_"," ")+" Sample"
    label_x = 0.15
    label_y = 0.85
    latex.DrawLatexNDC(label_x, label_y, label_text)


    canvas.cd()
    # Get the right TPad
    right_pad = ROOT.TPad("right_pad", "Right Pad", 0.65, 0, 1, 1)
    right_pad.SetLeftMargin(0)
    right_pad.Draw()
    right_pad.cd()

    # Divide the info pad into a 2x3 grid of text boxes
    right_pad.Divide(2, 3)
    textboxes=[]
    # Add text boxes to the info pad
    for i in range(6):
        textbox = ROOT.TPaveText(0, 0, 1, 1, "NDC")
        textbox.SetTextAlign(22)
        textbox.SetTextSize(0.08)
        textbox.SetFillColor(0)
        textbox.SetBorderSize(0)
        right_pad.cd(i+1)
        if(i<6):
            CHANNEL=channels[i].replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")
            textbox.AddText("#scale[1.2]{"+"#color["+str(colors[i])+"]"+"{"+"{}".format(CHANNEL)+"} channel cuts}")
            for cstr in cstrs[i]:
                textbox.AddText(cstr)
        textboxes.append(copy.deepcopy(textbox))
        textboxes[-1].Draw()


    # Update the TCanvas to show the histograms
    canvas.Draw()
    return copy.deepcopy(canvas)